In [1]:
using Pkg
Pkg.activate("rad_transfer"; shared=true)
using RadiativeTransfer
using RadiativeTransfer.CrossSection
using Pkg.Artifacts


 Activating environment at `~/.julia/environments/rad_transfer/Project.toml`


In [18]:
using NetCDF
using NCDatasets

In [12]:
reanalysis_file_path = "/export/data2/groupMembers/cchristo/reanalysis_3d/merra2/2015/01/MERRA2_400.tavg3_3d_asm_Nv.20150101.nc4"

"/export/data2/groupMembers/cchristo/reanalysis_3d/merra2/2015/01/MERRA2_400.tavg3_3d_asm_Nv.20150101.nc4"

In [66]:
size(T)

(576, 361, 72, 8)

In [67]:
# ds = ncread(reanalysis_file_path)
ds = Dataset(reanalysis_file_path)

times = ds["time"][:] 
lats = ds["lat"][:]
lons = ds["lon"][:]
T = ds["T"][:]
P = ds["PL"][:]
# (lon, lat, z, time)
var_array_sizes = size(T)



(576, 361, 72, 8)

In [81]:
# ncinfo(reanalysis_file_path)
desired_lat = 72.5796
desired_lon = -38.4588

# find index of nearest gridcell
lat_ind = argmin(broadcast(abs, lats .- desired_lat))
lon_ind = argmin(broadcast(abs, lons .- desired_lon))

# set time ind
time_ind = 1

1

In [83]:
T_prof = T[lat_ind,lon_ind, :,time_ind]
P_prof = P[]


72-element Array{Union{Missing, Float32},1}:
 202.64839f0
 199.50938f0
 202.49763f0
 209.20523f0
 216.67682f0
 222.78944f0
 229.02167f0
 232.51422f0
 235.90204f0
 240.63377f0
 245.91953f0
 252.45142f0
 259.57852f0
   ⋮
 280.10458f0
 279.96127f0
 279.77438f0
 279.4105f0
 278.9375f0
 278.70837f0
 278.9389f0
 279.52094f0
 280.2864f0
 280.83716f0
 281.77667f0
 279.28973f0

In [77]:
size(T_prof)

(72, 8)

In [5]:
# fetch('H2O_S',1,1,xmin,xmax)
# fetch('CO2_S',2,1,xmin,xmax)
# fetch('CH4_S',6,1,xmin,xmax)

## Open Profile

In [ ]:
hitran_data_CO2 = read_hitran("hitran_molec_id_2_CO2.tar", mol=2, iso=1, ν_min=400, ν_max=2100)
hitran_data_H2O = read_hitran("hitran_molec_id_1_H2O.tar", mol=1, iso=1, ν_min=400, ν_max=2100)
hitran_data_CH4 = read_hitran("hitran_molec_id_6_CH4.tar", mol=6, iso=1, ν_min=400, ν_max=2100)

In [3]:
hitran_data_CO2

UndefVarError: UndefVarError: hitran_data_CO2 not defined

In [3]:
model_voigt_GPU = make_hitran_model(hitran_data, Voigt(), architecture=Architectures.GPU())


UndefVarError: UndefVarError: hitran_data not defined